In [1]:
# Machine Learning: Logistic Regression
# Used this source: https://towardsdatascience.com/building-a-logistic-regression-in-python-301d27367c24


# Bollinger Bands

import pandas as pd
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

#portfolio start value is $100,000
portfolio_value = 100000

# Reading in aapl data
aapl = pd.read_csv("Assignment 3&4-data files/{}.csv".format('AAPL'), usecols=[0, 4],index_col=0)
helper_data = aapl.copy()

# Indicator 4: Bollinger Bands of length 20
# using this resource https://medium.com/python-data/setting-up-a-bollinger-band-with-python-28941e2fa300

# Finding 20 day moving average and standard deviation
twentyDay = aapl['Close'].rolling(window=20)
aapl['Daily Return'] = aapl['Close'] / aapl['Close'].shift(periods=-1) - 1
aapl['BB_Middle'] = twentyDay.mean() 
helper_data['20 Day Moving Avg'] = twentyDay.mean() 
# aapl['20 Day Std Dev'] = twentyDay.std()
helper_data['20 Day Std Dev'] = twentyDay.std()

# Finding upper and lower band (moving avg +- 2*std dev)
aapl['BB_Lower'] = aapl['BB_Middle'] - (helper_data['20 Day Std Dev'] * 2)
helper_data['BB_Lower'] = helper_data['20 Day Moving Avg'] - (helper_data['20 Day Std Dev'] * 2)
aapl['BB_Upper'] = aapl['BB_Middle'] + (helper_data['20 Day Std Dev'] * 2)
helper_data['BB_Upper'] = helper_data['20 Day Moving Avg'] + (helper_data['20 Day Std Dev'] * 2)


# bb_Sellsignal - Closing price crosses below upperband and was above upperband within 5 recent periods (0 or 1)
aapl['Sell_calc'] = aapl['Close'] - aapl['BB_Upper']
aapl['Buy_calc'] = aapl['Close'] - aapl['BB_Lower']
aapl['bb_Buysignal'] = 0
aapl['bb_Sellsignal'] = 0
aapl['Exitsignal'] = 0
aapl['bb_state'] = 0

# ExitSignal for BB bhappens when close proce is larger than the Upper band
for i in range(0, len(aapl)):
    if (aapl['BB_Upper'].iloc[i] < aapl['Close'].iloc[i]):
        aapl['Exitsignal'].iloc[i] = 1
#         aapl['Close'].iloc[i]


# Sell signal
for i in range(0, len(aapl)):
    if ((aapl['Sell_calc'].iloc[i] < 0) and (aapl['Sell_calc'].iloc[i-1] > 0) and (aapl['Sell_calc'].iloc[i-2] > 0) and (aapl['Sell_calc'].iloc[i-3] > 0) and (aapl['Sell_calc'].iloc[i-4] > 0) and (aapl['Sell_calc'].iloc[i-5] > 0)):
        aapl['bb_Sellsignal'].iloc[i]=1
#         aapl['bb_Sellsignal'].iloc[i] = aapl['Close'].iloc[i]+10

# Buy signal
for i in range(0, len(aapl)):
    if ((aapl['Buy_calc'].iloc[i] > 0) and (aapl['Buy_calc'].iloc[i-1] < 0) and (aapl['Buy_calc'].iloc[i-2] < 0) and (aapl['Buy_calc'].iloc[i-3] < 0) and (aapl['Buy_calc'].iloc[i-4] < 0) and (aapl['Buy_calc'].iloc[i-5] < 0)):
        aapl['bb_Buysignal'].iloc[i] = 1
#         aapl['bb_Buysignal'].iloc[i] = aapl['Close'].iloc[i]-10




# bb_state

# As confirmed with the TA:
# If there is no buy, sell or exit signal, the bb_state should be maintained (stay the same as the bb_state in the previous period).
# If in a long position and there's a buy signal, go into more long positions. 
# If in a long position and there's a sell signal, sell the long position and go into a short position.
# If in a short position and there's a buy signal, sell the short positions and go into a long position. 
# If in a short position and there's a sell signal, buy more short positions. 
# If in a short or long position and there's an exit signal, sell the positions. 

for i in range(1, len(aapl)):
    
#     maintain bb_state
    if(aapl['bb_Sellsignal'].iloc[i]==0 and aapl['bb_Buysignal'].iloc[i]==0 and aapl['Exitsignal'].iloc[i]==0):
        aapl['bb_state'].iloc[i]= aapl['bb_state'].iloc[i-1]
    
#     bb_Buysignal
    if(aapl['bb_Buysignal'].iloc[i]==1 and (aapl['bb_state'].iloc[i-1]==1 or aapl['bb_state'].iloc[i-1]==0)):
        # if you were long and you get a buy signal, buy more long positions
        # if you were neutral and you get a buy signal, buy long positions
        aapl['bb_state'].iloc[i]=1
    if(aapl['bb_Buysignal'].iloc[i]==1 and aapl['bb_state'].iloc[i-1]==-1):
        # if prev bb_state short and you get a buy sig, sell the short positions and buy long ones
        aapl['bb_state'].iloc[i]=1
        
#       bb_Sellsignal
    if(aapl['bb_Sellsignal'].iloc[i]==1 and aapl['bb_state'].iloc[i-1]==1):
         #if bb_Sellsignal 1 and previous bb_state was long, sell the long position and buy short
        aapl['bb_state'].iloc[i]=-1
    if(aapl['bb_Sellsignal'].iloc[i]==1 and (aapl['bb_state'].iloc[i-1]==0 or aapl['bb_state'].iloc[i-1]==-1)):
        # if you were neutral and you get a sell signal, buy short positions
        # if you were short and get a sell signal, buy more short positions
        aapl['bb_state'].iloc[i]=-1

#        exit signal 
    if(aapl['Exitsignal'].iloc[i]==1):
        aapl['bb_state'].iloc[i]= 0
    

    
# Filtering Exit Signals
aapl['ExitsignalFinal'] = 0

for i in range(1, len(aapl)):
    
    if(aapl['bb_state'].iloc[i] != aapl['bb_state'].iloc[i-1] and aapl['bb_state'].iloc[i]==0):
        aapl['ExitsignalFinal'].iloc[i]=1

/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
# RSI

window_length = 14
aapl['ROI'] = aapl['Close'] / aapl['Close'].shift(periods=-1) - 1

# Adjusted closing price of the stock
close = aapl['Close']

# Get the ROI 
roi = close.diff()
roi = roi[1:] 

# Separate positive and negative ROI values
positive_aapl = roi.copy()
negative_aapl = roi.copy()
positive_aapl[positive_aapl < 0] = 0
negative_aapl[negative_aapl > 0] = 0

# Calculate the Exponential Moving Average on positive and negative ROI
ewm_positive = positive_aapl.ewm(span = window_length).mean()
ewm_negative = (negative_aapl.abs()).ewm(span = window_length).mean()

# Calculate the RSI based on  Exponential Moving Average
ewm_RSI = ewm_positive / ewm_negative
ewm_rsi_score = 100.0 - (100.0 / (1.0 + ewm_RSI))

# Calculate the Simple Moving Average on positive and negative ROI
sma_positive = positive_aapl.rolling(window=window_length).mean()
sma_negative = (negative_aapl.abs()).rolling(window=window_length).mean()

# Calculate the RSI based on Simple Moving Average
sma_RSI = sma_positive / sma_negative
sma_rsi_score = 100.0 - (100.0 / (1.0 + sma_RSI))


# rsi_Sellsignal, rsi_Buysignal calculations based on EWM RSI only
aapl['EWM_RSI_Score']= ewm_rsi_score
aapl['SMA_RSI_Score']= sma_rsi_score
aapl['Sell_calc'] = ewm_rsi_score - 70
aapl['Buy_calc'] = ewm_rsi_score - 30
aapl['rsi_Sellsignal'] = 0
aapl['rsi_Buysignal'] = 0
aapl['Exitsignal'] = 0
aapl['rsi_state'] = 0

# ExitSignal for RSI happens when RSI score drops below 50 as per 
# https://www.dailyfx.com/forex/education/trading_tips/daily_trading_lesson/2018/05/10/3-trading-tips-for-RSI.html
for i in range(0, len(aapl)):
    if (aapl['EWM_RSI_Score'].iloc[i] < 50):
        aapl['Exitsignal'].iloc[i] = 1


# Sell signal calculations
for i in range(0, len(aapl)):
    if (aapl['Sell_calc'].iloc[i] < 0):
        if (aapl['Sell_calc'].iloc[i-1] > 0):
            if (aapl['Sell_calc'].iloc[i-2] > 0):
                if (aapl['Sell_calc'].iloc[i-3] > 0):
                    if (aapl['Sell_calc'].iloc[i-4] > 0):  
                        if (aapl['Sell_calc'].iloc[i-5] > 0): 
                            aapl['rsi_Sellsignal'].iloc[i] = 1


#rsi_Buysignal calculations
for i in range(0, len(aapl)):
    if (aapl['Buy_calc'].iloc[i] > 0):
        if (aapl['Buy_calc'].iloc[i-1] < 0):
            if (aapl['Buy_calc'].iloc[i-2] < 0):
                if (aapl['Buy_calc'].iloc[i-3] < 0):
                    if (aapl['Buy_calc'].iloc[i-4] < 0):  
                        if (aapl['Buy_calc'].iloc[i-5] < 0):  
                            aapl['rsi_Buysignal'].iloc[i] = 1
        
# rsi_state

# As confirmed with the TA:
# If there is no buy, sell or exit signal, the rsi_state should be maintained (stay the same as the rsi_state in the previous period).
# If in a long position and there's a buy signal, go into more long positions. 
# If in a long position and there's a sell signal, sell the long position and go into a short position.
# If in a short position and there's a buy signal, sell the short positions and go into a long position. 
# If in a short position and there's a sell signal, buy more short positions. 
# If in a short or long position and there's an exit signal, sell the positions. 

for i in range(1, len(aapl)):
    
#     maintain rsi_state
    if(aapl['rsi_Sellsignal'].iloc[i]==0 and aapl['rsi_Buysignal'].iloc[i]==0 and aapl['Exitsignal'].iloc[i]==0):
        aapl['rsi_state'].iloc[i]= aapl['rsi_state'].iloc[i-1]
    
#     rsi_Buysignal
    if(aapl['rsi_Buysignal'].iloc[i]==1 and (aapl['rsi_state'].iloc[i-1]==1 or aapl['rsi_state'].iloc[i-1]==0)):
        # if you were long and you get a buy signal, buy more long positions
        # if you were neutral and you get a buy signal, buy long positions
        aapl['rsi_state'].iloc[i]=1
    if(aapl['rsi_Buysignal'].iloc[i]==1 and aapl['rsi_state'].iloc[i-1]==-1):
        # if prev rsi_state short and you get a buy sig, sell the short positions and buy long ones
        aapl['rsi_state'].iloc[i]=1
        
#       rsi_Sellsignal
    if(aapl['rsi_Sellsignal'].iloc[i]==1 and aapl['rsi_state'].iloc[i-1]==1):
         #if rsi_Sellsignal 1 and previous rsi_state was long, sell the long position and buy short
        aapl['rsi_state'].iloc[i]=-1
    if(aapl['rsi_Sellsignal'].iloc[i]==1 and (aapl['rsi_state'].iloc[i-1]==0 or aapl['rsi_state'].iloc[i-1]==-1)):
        # if you were neutral and you get a sell signal, buy short positions
        # if you were short and get a sell signal, buy more short positions
        aapl['rsi_state'].iloc[i]=-1

#        exit signal 
    if(aapl['Exitsignal'].iloc[i]==1):
        aapl['rsi_state'].iloc[i]= 0
    

    
# Filtering Exit Signals
aapl['ExitsignalFinal'] = 0

for i in range(1, len(aapl)):
    
    if(aapl['rsi_state'].iloc[i] != aapl['rsi_state'].iloc[i-1] and aapl['rsi_state'].iloc[i]==0):
        aapl['ExitsignalFinal'].iloc[i]=1
        
# Calculating Shares and Portfolio Value
aapl['Shares'] = 0

aapl['Portfolio Value'] = 0
share_num = 20

for i in range(1, len(aapl)):
    
    if((aapl['rsi_Buysignal'].iloc[i] == 0) and (aapl['rsi_Sellsignal'].iloc[i] ==0) and (aapl['Exitsignal'].iloc[i] ==0)):
        aapl['Shares'].iloc[i]= aapl['Shares'].iloc[i-1]
    
    #buy condition when long or no position
    if(aapl['rsi_Buysignal'].iloc[i] ==1 and ((aapl['rsi_state'].iloc[i-1] == 1) or (aapl['rsi_state'].iloc[i-1] == 0))):
        shares_afford_long = portfolio_value/aapl['Close'].iloc[i]
        if (shares_afford_long >= share_num):
            # if you can afford 10 shares, buy 10
            aapl['Shares'].iloc[i]= aapl['Shares'].iloc[i-1] + share_num
            portfolio_value = portfolio_value - (aapl['Close'].iloc[i]*share_num)
        else:
            # if you can't afford 10 shares, buy as many as you can afford
            aapl['Shares'].iloc[i]= aapl['Shares'].iloc[i-1] + shares_afford_long
            portfolio_value = portfolio_value - (aapl['Close'].iloc[i]*shares_afford_long)
    aapl['Portfolio Value'].iloc[i] = portfolio_value
    
    #buy condition when short 
    if((aapl['rsi_Buysignal'].iloc[i] ==1) and (aapl['rsi_state'].iloc[i-1] == -1)):
        # Selling all short positions, so number of shares becomes 0
        portfolio_value = portfolio_value + aapl['Shares'].iloc[i-1]*aapl['Close'].iloc[i]
        aapl['Shares'].iloc[i] = 0

        shares_afford_long = portfolio_value/aapl['Close'].iloc[i]
        if (shares_afford_long >= share_num):
            # if you can afford 10 shares, buy 10
            aapl['Shares'].iloc[i]= share_num
            portfolio_value = portfolio_value - (aapl['Close'].iloc[i]*share_num)
        else:
            # if you can't afford 10 shares, buy as many as you can afford
            aapl['Shares'].iloc[i]= shares_afford_long
            portfolio_value = portfolio_value - (aapl['Close'].iloc[i]*shares_afford_long)
    aapl['Portfolio Value'].iloc[i] = portfolio_value

    # sell condition when long
    if((aapl['rsi_Sellsignal'].iloc[i] ==1) and aapl['rsi_state'].iloc[i-1] == 1):
        
        # Selling all short positions, so number of shares becomes 0
        portfolio_value = portfolio_value - aapl['Shares'].iloc[i-1]*aapl['Close'].iloc[i]
        aapl['Shares'].iloc[i] = 0

        shares_afford_short = portfolio_value/aapl['Close'].iloc[i]
        if (shares_afford_short >= share_num):
            # if you can afford 10 shares, buy 10

            aapl['Shares'].iloc[i]= - share_num
            portfolio_value = portfolio_value + share_num*aapl['Close'].iloc[i]
        else:
            # if you can't afford 10 shares, buy as many as you can afford
            aapl['Shares'].iloc[i]= - shares_afford_short
            portfolio_value = portfolio_value + shares_afford_short*aapl['Close'].iloc[i]
    aapl['Portfolio Value'].iloc[i] = portfolio_value
    
    # sell condition when short or no position
    if((aapl['rsi_Sellsignal'].iloc[i] ==1) and ((aapl['rsi_state'].iloc[i-1] == -1) or (aapl['rsi_state'].iloc[i-1] == 0))):
        shares_afford_short = portfolio_value / aapl['Close'].iloc[i]
        if (shares_afford_short >= share_num):
            # if you can afford 10 shares, buy 10
            aapl['Shares'].iloc[i]= aapl['Shares'].iloc[i-1]-10
            portfolio_value = portfolio_value + share_num*aapl['Close'].iloc[i]
        else:
            # if you can't afford 10 shares, buy as many as you can afford
            aapl['Shares'].iloc[i]= aapl['Shares'].iloc[i-1]-shares_afford_short
            portfolio_value = portfolio_value + shares_afford_short*aapl['Close'].iloc[i]
    aapl['Portfolio Value'].iloc[i] = portfolio_value

                
    # exit condition
    if(aapl['ExitsignalFinal'].iloc[i] ==1 and aapl['rsi_state'].iloc[i-1] == 1):
        portfolio_value = portfolio_value + aapl['Shares'].iloc[i-1]*aapl['Close'].iloc[i]
        aapl['Shares'].iloc[i]= 0
    aapl['Portfolio Value'].iloc[i] = portfolio_value
        
    if(aapl['ExitsignalFinal'].iloc[i] ==1 and aapl['rsi_state'].iloc[i-1] == -1):
        portfolio_value = portfolio_value + aapl['Shares'].iloc[i-1]*aapl['Close'].iloc[i]
        aapl['Shares'].iloc[i]= 0
    aapl['Portfolio Value'].iloc[i] = portfolio_value

In [4]:
aapl.head()

,Close,Daily Return,BB_Middle,BB_Lower,BB_Upper,Sell_calc,Buy_calc,bb_Buysignal,bb_Sellsignal,Exitsignal,bb_state,ExitsignalFinal,ROI,EWM_RSI_Score,SMA_RSI_Score,rsi_Sellsignal,rsi_Buysignal,rsi_state,Shares,Portfolio Value
Date,,,,,,,,,,,,,,,,,,,,
2017-01-03,116.150002,0.001121,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0.001121,NaN,NaN,0,0,0,0,0.0
2017-01-04,116.019997,-0.005060,NaN,NaN,NaN,-70.000000,-30.000000,0,0,1,0,0,-0.005060,0.000000,NaN,0,0,0,0,100000.0
2017-01-05,116.610001,-0.011025,NaN,NaN,NaN,13.965418,53.965418,0,0,0,0,0,-0.011025,83.965418,NaN,0,0,0,0,100000.0
2017-01-06,117.910004,-0.009076,NaN,NaN,NaN,24.884819,64.884819,0,0,0,0,0,-0.009076,94.884819,NaN,0,0,0,0,100000.0
2017-01-09,118.989998,-0.001007,NaN,NaN,NaN,26.905102,66.905102,0,0,0,0,0,-0.001007,96.905102,NaN,0,0,0,0,100000.0


In [37]:
aapl['False/True Signal'] = False

# use Daily return as true/false
for i in range(1,len(aapl)):   
    #if previous bb_state is 1 and the return is positive, you are in the right position
#     if(aapl['Daily Return'].iloc[i] > 0 and (aapl['bb_state'].iloc[i-1] == 1 or aapl['rsi_state'].iloc[i-1]==1)):
#         aapl['False/True Signal'].iloc[i] = True
#     if(aapl['Daily Return'].iloc[i] < 0 and (aapl['bb_state'].iloc[i-1] == -1 or aapl['rsi_state'].iloc[i-1]==-1)):
#         aapl['False/True Signal'].iloc[i] = True  
    if(aapl['Daily Return'].iloc[i] > 0):
        aapl['False/True Signal'].iloc[i] = True

   
   

relevant_cols = aapl[['Close','bb_Buysignal','bb_Sellsignal','rsi_Buysignal','rsi_Sellsignal', 'False/True Signal']].copy()
relevant_cols.head()

In [55]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


def load_data(path, header):
    marks_df = pd.read_csv(path, header=header)
    return marks_df


# X = feature values
X = relevant_cols.iloc[:, [1,2,3,4]]

# y = target values 
y = relevant_cols.iloc[:, 5]

# filter out the applicants that got admitted
true_rows = relevant_cols.loc[y == True]

# filter out the applicants that din't get admission
false_rows = relevant_cols.loc[y == False]

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

# X = np.c_[np.ones((X.shape[0], 1)), X]
# y = y[:, np.newaxis]

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y,predicted_classes)
parameters = model.coef_


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [66]:

# only add prediction to testing set
aapl['Prediction'][len(X_train):] = y_pred
aapl.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Close,Daily Return,BB_Middle,BB_Lower,BB_Upper,Sell_calc,Buy_calc,bb_Buysignal,bb_Sellsignal,Exitsignal,...,ROI,EWM_RSI_Score,SMA_RSI_Score,rsi_Sellsignal,rsi_Buysignal,rsi_state,Shares,Portfolio Value,False/True Signal,Prediction
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,116.150002,0.001121,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0.001121,NaN,NaN,0,0,0,0,0.0,False,0
2017-01-04,116.019997,-0.005060,NaN,NaN,NaN,-70.000000,-30.000000,0,0,1,...,-0.005060,0.000000,NaN,0,0,0,0,100000.0,False,0
2017-01-05,116.610001,-0.011025,NaN,NaN,NaN,13.965418,53.965418,0,0,0,...,-0.011025,83.965418,NaN,0,0,0,0,100000.0,False,0
2017-01-06,117.910004,-0.009076,NaN,NaN,NaN,24.884819,64.884819,0,0,0,...,-0.009076,94.884819,NaN,0,0,0,0,100000.0,False,0
2017-01-09,118.989998,-0.001007,NaN,NaN,NaN,26.905102,66.905102,0,0,0,...,-0.001007,96.905102,NaN,0,0,0,0,100000.0,False,0


In [67]:
accuracy

0.54611211573236895

In [68]:
cm

array([[76,  1],
       [60,  2]])

In [69]:
aapl.to_csv('aapl-predicted.csv')